In [ ]:
import numpy as np
from math import *
from random import *
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import cm

def pricer_MC (n,s,r,sigma,T,f) :
    epsilon = np.random.normal(0,1,n)
    somme = 0
    for i in range (1, n+1) :
        val = s*np.exp((r-(sigma**2)/2)*T)+sigma*(T**(1/2))*epsilon[i-1]
        somme = somme + f(val)
    return (exp(-r*T)/n)*somme

Y=[]
for k in range(1,11):
    n=k * 10 ** 5
    f=lambda x:max(x-100,0)
    Y.append(pricer_MC(n,100,0.01,0.1,1,f))

print("Question 13\n")
plt.plot(range(1,11),Y)
plt.show()

def F (x) :
    return stats.norm.cdf(x)

def put_BS (s,r,sigma,T,K) :
    d1 = (1/(sigma*(T**(1/2))))*(np.log(s/K)+(r+(sigma**2)/2)*T)
    d2 = d1-sigma * (T**(1/2))
    prix_BS = -s*F(-d1) + K*np.exp(-r*T)*F(-d2)
    return prix_BS

print("Question 16\n")
print("Pricer par formule fermée:",put_BS(100,0.01,0.1,1,90),"\n")

print("Question 17\n")
def pricer_MC (n,s,r,sigma,T,f) :
    epsilon = np.random.normal(0,1,n)
    somme = 0
    for i in range (1, n+1) :
        val = s*np.exp((r-(sigma**2)/2)*T+sigma*(T**(1/2))*epsilon[i-1])
        somme = somme + f(val)
    return (exp(-r*T)/n)*somme

def F (x) :
    return stats.norm.cdf(x)

Y=[]
Z=[]
for k in range(1,11):
    n=k * 10 ** 5
    f=lambda x:max(90-x,0)
    Y.append(pricer_MC(n,100,0.01,0.1,1,f))
    Z.append(put_BS(100,0.01,0.1,1,90))

plt.plot(range(1,11),Y,label="pricer_MC")
plt.plot(range(1,11),Z,label="put_BS")
plt.legend()
plt.show()

print("Question 18")
k_list=range(1,11)
T_list=[1/12,1/6,1/4,1/3,1/2,1]
prix=[]
for k in k_list:
    L=[]
    for T in T_list:
        L.append(put_BS(20*k,0.01,0.1,T,100))
    prix.append(L)
prix=np.array(prix)

fig = plt.figure()
k_list,T_list=np.meshgrid(k_list,T_list)
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('k')
ax.set_ylabel('T')
ax.set_zlabel('Prix')
surf = ax.plot_surface(k_list, T_list, prix.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

print("Question 19\n")
def pricer_2(N,r_N,h_N,b_N,s,f) :
    def q_N(r_N, b_N, h_N):
        return (r_N-b_N)/(h_N-b_N)
    qN=q_N(r_N,b_N,h_N)
    l1=[]
    for k in range (0,N+1) :
        e = f(s*(1+h_N)**(N-k)*(1+b_N)**k)
        l1.append(e)
    for n in range (N,0,-1) :
        l2=[]
        for k in range (0,n):
            e = (l1[k]*qN + l1[k+1]*(1-qN))/(1+r_N)
            l2.append(e)
        l1 = l2
    return l1[0]

Y=[]
Z=[]
for k in range(1,101):
    r_N=0.03*1/(10*k)
    h_N=(1+r_N)*exp(0.2*sqrt(1/(10*k)))-1
    b_N=(1+r_N)*exp(-0.2*sqrt(1/(10*k)))-1
    Y.append(pricer_2(10*k,r_N,h_N,b_N,100,lambda x : max(100-x,0)))
    Z.append(put_BS(100,0.03,0.2,1,100))

plt.clf()
plt.plot(range(1,101),Y,label="pricer_2")
plt.plot(range(1,101),Z,label="put_BS")
plt.legend()
plt.show()

print("Question 20")
import scipy.linalg

def p_t_x_min(K, T, M, r, x_min, m):
    delta_T = T/M
    return(K*exp(-r*m*delta_T)-exp(x_min))

def theta_fonction(theta, K, r, sigma, T, x_min, x_max, N, M):
    h = (x_max - x_min)/N
    delta_t = T/M
    A = -(theta*sigma**2)/(2*h**2) + (r-(sigma**2)/(2))*((theta)/(2*h))
    B = -(1)/(delta_t) + (theta*sigma**2)/(h**2) + r
    C = -(theta*sigma**2)/(2*h**2) - (r-(sigma**2)/(2))*((theta)/(2*h))
    D = -((1-theta)*sigma**2)/(2*h**2) + (r-(sigma**2)/(2))*((1-theta)/(2*h))
    E = (1)/(delta_t) + ((1-theta)*sigma**2)/(h**2)
    F = -((1-theta)*sigma**2)/(2*h**2) - (r-(sigma**2)/(2))*((1-theta)/(2*h))
    U = np.zeros((N-1,N-1))
    V = np.zeros((N-1,N-1))
    W = np.zeros((N-1,1))
    P = np.zeros((N-1,1))

    for i in range(0, N-1):
        P[i,0] = max(0, K - exp(x_min + (i+1)*h))
        U[i,i] = E
        V[i,i] = -B
        if i>0:
            U[i,i-1] = D
            V[i,i-1] = -A
        if i<N-2:
            U[i,i+1] = F
            V[i,i+1] = -C

    (Z,l,u) = scipy.linalg.lu(U)

    for m in range(0, M):
        W[0,0] = -D*p_t_x_min(K, T, M, r, x_min, m+1)-A*p_t_x_min(K, T, M, r, x_min, m)
        X = np.dot(V, P) + W
        Y = np.linalg.solve(l, X)
        P = np.linalg.solve(u, Y)

    return P

h = (log(2) - log(0.4))/100
X = theta_fonction(1, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, 100)
X2 = theta_fonction(1, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, 400)
Y = theta_fonction(0, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, 100)
Z = theta_fonction(0.5, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, 100)
W = np.linspace(log(0.4)+h,log(2)-h,99)

print("Méthode explicite (cas N=M)")
plt.plot(W,X)
plt.show()
print("Méthode explicite (cas N=100 et M=400)")
plt.plot(W,X2)
plt.show()
print("Méthode implicite")
plt.plot(W,Y)
plt.show()
print("Méthode de Crank-Nicolson")
plt.plot(W,Z)
plt.show()

print("Evaluation des erreurs")
print("Méthode explicite")
x_list=W[:]
lower_bound = 50
upper_bound = 250
prix=np.zeros((len(x_list),len(range(lower_bound,upper_bound))))

value_put_BS = np.zeros((len(x_list),1))
for i in range(len(x_list)):
    value_put_BS[i,0]=put_BS(exp(x_list[i]),0.015,0.21,1,0.9)

for j in range(lower_bound,upper_bound):
    vect=np.abs(theta_fonction(1, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, j) - value_put_BS)/value_put_BS
    for i in range(len(x_list)):
        prix[i,j-lower_bound]=vect[i,0]

fig = plt.figure()
x_list,M_list=np.meshgrid(x_list,range(lower_bound,upper_bound))
ax = fig.add_subplot(projection='3d')
surf = ax.plot_surface(x_list, M_list, prix.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('x')
ax.set_ylabel('M')
ax.set_zlabel('Prix')
plt.show()

print("Méthode implicite")
x_list=W[:]
prix=np.zeros((len(x_list),len(range(lower_bound,upper_bound))))
for j in range(lower_bound,upper_bound):
    vect=np.abs(theta_fonction(0, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, j) - value_put_BS)/value_put_BS
    for i in range(len(x_list)):
        prix[i,j-lower_bound]=vect[i,0]

fig = plt.figure()
x_list,M_list=np.meshgrid(x_list,range(lower_bound,upper_bound))
ax = fig.add_subplot(projection='3d')
surf = ax.plot_surface(x_list, M_list, prix.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('x')
ax.set_ylabel('M')
ax.set_zlabel('Prix')
plt.show()
print("Méthode de Crank-Nicolson")
x_list=W[:]
prix=np.zeros((len(x_list),len(range(lower_bound,upper_bound))))
for j in range(lower_bound,upper_bound):
    vect=np.abs(theta_fonction(0.5, 0.9, 0.015, 0.21, 1, log(0.4), log(2), 100, j) - value_put_BS)/value_put_BS
    for i in range(len(x_list)):
        prix[i,j-lower_bound]=vect[i,0]

fig = plt.figure()
x_list,M_list=np.meshgrid(x_list,range(lower_bound,upper_bound))
ax = fig.add_subplot(projection='3d')
surf = ax.plot_surface(x_list, M_list, prix.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('x')
ax.set_ylabel('M')
ax.set_zlabel('Prix')
plt.show()